In [1]:
import os,sys

In [2]:
from pathlib import Path

In [8]:
os.getcwd()
os.chdir("..")

In [9]:
ls

 Volume in drive C is Windows
 Volume Serial Number is 0695-0DCC

 Directory of c:\project\credit_card_fault

23-04-2023  22:16    <DIR>          .
06-04-2023  20:43    <DIR>          ..
23-04-2023  22:16    <DIR>          .dvc
23-04-2023  22:09               142 .dvcignore
06-04-2023  21:18    <DIR>          .github
10-04-2023  20:43             1,960 .gitignore
19-04-2023  20:48    <DIR>          .pytest_cache
25-04-2023  11:57    <DIR>          artifact
25-04-2023  17:35    <DIR>          configs
25-04-2023  11:57             2,559 dvc.lock
25-04-2023  11:56             1,454 dvc.yaml
18-04-2023  22:20    <DIR>          env
06-04-2023  22:47               254 init_setup.sh
06-04-2023  20:51             1,087 LICENSE
25-04-2023  12:51    <DIR>          logs
23-04-2023  22:11             1,595 params.yaml
06-04-2023  21:36               226 pyproject.toml
06-04-2023  20:40                12 README.md
06-04-2023  22:29             2,706 requirements.txt
06-04-2023  23:27               

In [3]:
from creditcard.entity import *
from creditcard.components import *
from creditcard.constants import *
from creditcard.utils import *

In [10]:
ROOT_DIR = os.getcwd()  # to get current working directory

# config constants
CONFIG_DIR = os.path.join(ROOT_DIR, "configs")
CONFIG_FILE_NAME = "config.yaml"
CONFIG_FILE_PATH = os.path.join(CONFIG_DIR, CONFIG_FILE_NAME)
print(CONFIG_FILE_PATH)

c:\project\credit_card_fault\configs\config.yaml


In [11]:
config_info = read_yaml(path_to_yaml = Path(CONFIG_FILE_PATH))

2023-04-27 10:25:53.769 | INFO     | creditcard.utils.common:read_yaml:34 - yaml file: c:\project\credit_card_fault\configs\config.yaml loaded successfully


In [12]:
pipeline_config = config_info.training_pipeline_config
print(pipeline_config)

{'artifact_dir': 'artifact', 'experiment_code': 'Development', 'pipeline_name': 'CreditCard', 'random_state': 1961}


In [14]:
artifact_dir = pipeline_config.artifact_dir
experiment_code = pipeline_config.experiment_code


In [15]:
from creditcard.config import *

In [17]:
SCHEMA_FILE_NAME = "schema.yaml"
SCHEMA_FILE_PATH = os.path.join(CONFIG_DIR,SCHEMA_FILE_NAME)
print(SCHEMA_FILE_PATH)

c:\project\credit_card_fault\configs\schema.yaml


In [19]:
data_validation_config_info = config_info.data_validation_config
validated_train_file_name = data_validation_config_info.validated_train_file
data_validated_artifact_dir = Path(os.path.join(artifact_dir,data_validation_config_info.data_validation_dir))
validated_train_file_path = os.path.join(data_validated_artifact_dir, validated_train_file_name)
print(validated_train_file_path)

artifact\stage02_data_validation\validated_train.pkl


In [20]:
model_trainer_config_info = config_info.model_trainer_config
model_trainer_artifact_dir_name = model_trainer_config_info.model_trainer_dir
model_trainer_artifact_dir = os.path.join(artifact_dir, model_trainer_artifact_dir_name)
model_report_dir_name = model_trainer_config_info.model_reports_dir


In [22]:

trained_model_path = os.path.join(model_trainer_artifact_dir, "best_model", "best_model.pkl")
print(trained_model_path)

artifact\stage04_model_training\best_model\best_model.pkl


In [23]:
model_report_dir = os.path.join(model_trainer_artifact_dir, model_report_dir_name)

In [24]:
data_transformation_config_info = config_info.data_transformation_config

In [25]:
data_transformation_dir_name = data_transformation_config_info.data_transformation_dir
data_transformation_dir = os.path.join(artifact_dir, data_transformation_dir_name)

In [26]:
preprocessed_object_dir = data_transformation_config_info.preprocessing_object_dir
preprocessed_object_name = data_transformation_config_info.preprocessing_object_file_name
preprocessed_object_file_path = os.path.join(data_transformation_dir, preprocessed_object_dir,
                                                         preprocessed_object_name)

In [27]:
create_directories([os.path.dirname(trained_model_path), model_report_dir])

2023-04-27 10:41:13.726 | INFO     | creditcard.utils.common:create_directories:74 - created directory at: artifact\stage04_model_training\best_model
2023-04-27 10:41:13.732 | INFO     | creditcard.utils.common:create_directories:74 - created directory at: artifact\stage04_model_training\model_eval_reports


In [29]:
from creditcard.utils import *

In [34]:
def get_model_trainer_config(self, model_config_file_path: str, schema_file_path: str , data_validation_config_info : DataValidationConfig , 
                                 data_transformation_config_info : DataTransformationConfig) -> ModelTrainerConfig:
       
            pipeline_config = self.pipeline_config
            artifact_dir = pipeline_config.artifact_dir
            experiment_code = pipeline_config.experiment_code
            schema_file_path = data_validation_config_info.schema_file_path
            validated_train_file_path = data_validation_config_info.data_validated_train_file_path
            model_trainer_config_info = self.config_info.model_trainer_config
            model_trainer_artifact_dir_name = model_trainer_config_info.model_trainer_dir
            model_trainer_artifact_dir = os.path.join(artifact_dir, model_trainer_artifact_dir_name)
            model_report_dir_name = model_trainer_config_info.model_reports_dir

            trained_model_path = os.path.join(model_trainer_artifact_dir, "best_model", "best_model.pkl")
            model_report_dir = os.path.join(model_trainer_artifact_dir, model_report_dir_name)
            preprocessed_object_file_path = data_transformation_config_info.preprocessed_object_file_path

            create_directories([os.path.dirname(trained_model_path), model_report_dir])

            model_trainer_config = ModelTrainerConfig(model_config_file_path=model_config_file_path,
                                                      base_accuracy=model_trainer_config_info.base_accuracy,
                                                      trained_model_file_path=trained_model_path,
                                                      model_report_dir=model_report_dir,
                                                      preprocessed_object_file_path=preprocessed_object_file_path,
                                                      schema_file_path=schema_file_path,
                                                      eval_difference=model_trainer_config_info.eval_difference,
                                                      eval_param=model_trainer_config_info.eval_param,
                                                      experiment_id=experiment_code,
                                                      to_train_data_path=validated_train_file_path, )

            return model_trainer_config

In [39]:
from pydantic import BaseModel, DirectoryPath, FilePath


class DataIngestionArtifact(BaseModel):
    train_file_path: FilePath
    test_file_path: FilePath


class DataValidationArtifact(BaseModel):
    schema_file_path: FilePath
    report_file_dir: DirectoryPath
    is_validated: bool


class DataTransformationArtifact(BaseModel):
    preprocessed_object_path: FilePath
    
class ModelTrainerArtifact(BaseModel):
    pass

In [35]:
print(model_trainer_config_info)

{'base_accuracy': 0.7, 'eval_difference': 0.6, 'eval_param': 'roc_auc', 'model_factory_file_name': 'model_factory.yaml', 'model_reports_dir': 'model_eval_reports', 'model_trainer_dir': 'stage04_model_training', 'models': ['GBT', 'RF', 'LGBM', 'CATBOOST']}


In [40]:
from pathlib import Path

from pydantic import BaseModel, DirectoryPath, FilePath


class DataIngestionConfig(BaseModel):
    dataset_download_id: str
    raw_data_file_path: Path
    ingested_train_file_path: Path
    ingested_test_data_path: Path
    random_state: int


class TrainingPipelineConfig(BaseModel):
    artifact_dir: DirectoryPath
    pipeline_name: str
    experiment_code: str
    training_random_state: int


class DataValidationConfig(BaseModel):
    experiment_code: str
    data_validated_artifact_dir: DirectoryPath
    schema_file_path: FilePath
    report_file_dir: Path
    data_validated_test_collection: str
    data_validated_train_collection: str
    train_data_file: FilePath
    test_data_file: FilePath


class DataTransformationConfig(BaseModel):
    data_validated_train_collection: str
    schema_file_path: FilePath
    random_state: int
    preprocessed_object_file_path: Path
    to_train_collection: str
    to_test_collection: str
    
class ModelTrainerConfig(BaseModel):
    model_config_file_path : Path
    base_accuracy : int
    trained_model_file_path : Path 
    model_report_dir : DirectoryPath
    preprocessed_object_file_path: FilePath
    to_train_collection: str
    to_test_collection: str
    schema_file_path : FilePath
    model_report_dir : DirectoryPath
    eval_difference : int
    eval_param  : str
    experiment_id : str
    validated_collection : str

In [41]:
model_trainer_config_info: ModelTrainerConfig

In [42]:
model_trainer_config_info = model_trainer_config_info

In [44]:
from pathlib import Path

from pydantic import BaseModel, DirectoryPath, FilePath

class MetricReportArtifact(BaseModel):
        experiment_id : str 
        model_name : str 
        model_obj : object
        report : object

class MetricEvalArtifact(BaseModel):
        best_model : object
        best_train_eval_param  : int
        best_test_eval_param : int
        best_eval_param_difference : int 
        best_model_name  : str
        best_model_report : object

In [46]:
schema_data = read_yaml(path_to_yaml=Path(SCHEMA_FILE_PATH))

2023-04-27 11:00:24.950 | INFO     | creditcard.utils.common:read_yaml:34 - yaml file: c:\project\credit_card_fault\configs\schema.yaml loaded successfully


In [47]:
data_path = r"C:\project\credit_card_fault\artifact\stage02_data_validation\validated_train.pkl"

In [48]:
import pandas as pd

In [49]:
raw_data = pd.read_pickle(data_path)

In [50]:
raw_data

,ID,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,...,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default
0,6637,80000.0,2,3,1,41,2,2,2,2,...,61129.0,62348.0,61193.0,0.0,5000.0,2400.0,2341.0,0.0,2439.0,1
1,29302,50000.0,1,1,2,25,0,0,0,2,...,48174.0,48731.0,49655.0,4000.0,4000.0,2000.0,2000.0,2200.0,1700.0,0
2,2164,20000.0,1,1,2,30,0,0,2,0,...,19391.0,19120.0,19158.0,6500.0,84.0,7171.0,1000.0,1000.0,2000.0,0
3,6442,70000.0,2,2,2,24,0,0,0,0,...,64457.0,66021.0,67267.0,3167.0,2521.0,2400.0,2600.0,2614.0,2376.0,0
4,23135,210000.0,2,3,1,41,0,0,-1,-1,...,-649.0,-649.0,-649.0,2000.0,17864.0,569.0,0.0,0.0,1000.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23995,854,140000.0,2,2,2,23,0,0,0,0,...,43455.0,43980.0,44614.0,2100.0,2027.0,1800.0,1600.0,1750.0,1700.0,0
23996,25278,170000.0,2,1,2,27,-1,0,0,0,...,38330.0,39602.0,39922.0,1650.0,4500.0,4000.0,4000.0,3000.0,3000.0,0
23997,6002,500000.0,2,1,2,37,-1,-1,-1,-1,...,0.0,1450.0,2668.0,1625.0,517.0,0.0,1450.0,2668.0,1480.0,0
23998,21747,20000.0,2,1,2,24,0,0,2,-1,...,6572.0,5016.0,2508.0,2778.0,0.0,6572.0,5016.0,0.0,0.0,0


In [51]:
x_train = raw_data.drop(schema_data.base_model_features_to_drop, axis=1)

In [52]:
y_train = raw_data[schema_data.target_column]

In [53]:
y_train

0        1
1        0
2        0
3        0
4        0
        ..
23995    0
23996    0
23997    0
23998    0
23999    1
Name: default, Length: 24000, dtype: int64

In [54]:
preprocessing_obj_path =r"C:\project\credit_card_fault\artifact\stage03_data_transformation\preprocessing\preprocessing_obj.pkl"

In [56]:
preprocessing_obj = load_object(file_path=Path(preprocessing_obj_path))

In [58]:
x_processed = preprocessing_obj.transform(x_train)

In [59]:
import numpy

In [60]:
data_to_train = numpy.c_[x_processed, y_train]

In [61]:
data_to_train

array([[0., 1., 0., ..., 0., 2., 1.],
       [1., 0., 0., ..., 0., 2., 0.],
       [1., 0., 0., ..., 0., 3., 0.],
       ...,
       [0., 0., 0., ..., 0., 1., 0.],
       [1., 0., 0., ..., 0., 3., 0.],
       [0., 1., 0., ..., 0., 1., 1.]])

In [62]:
trained_model_columns = schema_data.base_model_trained_columns

In [64]:
model_factory_config_file_path = r"C:\project\credit_card_fault\configs\model_factory.yaml"

In [65]:
Model_factory_config_info = read_yaml(Path(model_factory_config_file_path))

2023-04-27 11:32:15.874 | INFO     | creditcard.utils.common:read_yaml:34 - yaml file: C:\project\credit_card_fault\configs\model_factory.yaml loaded successfully


In [66]:
model_keys = Model_factory_config_info.models.keys()

In [67]:
model_keys

dict_keys(['GBT', 'LGBM', 'RF', 'CATBOOST'])